In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
def loadDataWithResizeShape(dataset,resizeShape,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                                   image_size=resizeShape, batch_size=32)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                                   validation_split=split,subset='both',
                                                                   image_size=resizeShape, batch_size=32)

In [3]:
def createTransferableModel(base_model, numberOfNeuronsPerFCLayer, adamLearningRate, L2Rate):
    model = keras.Sequential()
    model.add(base_model)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
    model.add(keras.layers.Dense(1,activation='sigmoid'))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer=adamOptimizer,loss='binary_crossentropy', metrics=basicUtil.f1_score)
    return model

In [4]:
def createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScores,devScores):
    modelParameters = dict()
    modelParameters['n_NeuronsPerFCLayers'] = n_NeuronsPerFCLayers
    modelParameters['n_Epochs'] = n_Epochs
    modelParameters['adamLearningRates'] = adamLearningRates
    modelParameters['L2Rates'] = L2Rates
    modelParameters['trainScore'] = trainScores
    modelParameters['devScore'] = devScores

    modelParametersDF = pd.DataFrame(modelParameters, columns=modelParameters.keys())
    return modelParametersDF

In [5]:
def main():
    imageShape = (180,180)
    inputShape = [imageShape[0],imageShape[1],3]
    train = loadDataWithResizeShape("training",imageShape)
    dev,test = loadDataWithResizeShape("testing",imageShape,.5)
    
    base_model = VGG16(weights="imagenet", include_top=False,input_shape=inputShape,
                      classifier_activation = None,classes = len(train.class_names))
    for layer in base_model.layers:
        layer.trainable = False

    possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
    possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)
    adamLearningRateCriticalPoints = (1e-4,1e-2)
    L2CriticalPoints = (1e-2,1e3) 

    n_NeuronsPerFCLayers = []
    n_Epochs = []
    adamLearningRates = []
    L2Rates = []
    trainScores = []
    devScores = []

    trial = 0
    bestDevScore = 0
    oversizedNeuronNumbers = []
    oversizedEpochNumbers = []
    
    while trial < 100:
        numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
        numberOfEpochs = choose(possibleEpochs)
        
        adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
        L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
        model = createTransferableModel(base_model, numberOfNeuronsPerFCLayer, adamLearningRate, L2Rate)
        
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/VGG16Trials/vgg16_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 60:
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            if (devScore > 0.91) and (devScore > bestDevScore):
                testScore = model.evaluate(test)[1]
                model_path = f'../Models/best_vgg16_model_.h5'
                model.save(model_path)
                bestModelParams = {
                    'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                    'n_Epochs' : int(numberOfEpochs),
                    'adamLearningRates' : adamLearningRate,
                    'L2Rates' : L2Rate,
                    'modelSize' : model_size,
                    'trainScore': trainScore,
                    'devScore': devScore,
                    'testScore': testScore
                }
                with open('../Models/best_vgg16_model_params.json', 'w') as f:
                    json.dump(bestModelParams, f)
                bestDevScore = devScore       

            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            n_Epochs.append(numberOfEpochs)
            
            adamLearningRates.append(adamLearningRate)
            L2Rates.append(L2Rate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],[adamLearningRate],
                                                          [L2Rate],[np.nan],[np.nan])
            display(failedTrial)
            oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
            oversizedEpochNumbers.append(numberOfEpochs)
            if len(oversizedNeuronNumbers) >= 3:
                possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
                possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
                oversizedNeuronNumbers = []
                oversizedEpochNumbers = []
            
        if (trial % 10 == 9): 
            modelParametersDF = createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScores,devScores)
            modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
            display(modelParametersDF)
            
            top5 = modelParametersDF[0:5]
            possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
        
            possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
            adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
            L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

            n_NeuronsPerFCLayers = []
            n_Epochs = []
            adamLearningRates = []
            L2Rates = []
            trainScores = []
            devScores = []

            if bestDevScore > 0.91:
                trial = 101
                
    basicUtil.displayFinalResults('../Models/best_vgg16_model_params.json')
    

In [6]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-09-22 16:24:28.928395: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-09-22 16:24:28.928541: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-09-22 16:24:29.149565: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 95s 1s/step - loss: 0.7060 - f1_score: 0.0000e+00
devScore


2024-09-22 16:40:28.273344: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]
2024-09-22 16:40:28.273496: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 13s 1s/step - loss: 0.7049 - f1_score: 0.0000e+00
concluding trial  0
trainScore
75/75 [==============================] - 92s 1s/step - loss: 1.6120 - f1_score: 0.5171
devScore
10/10 [==============================] - 12s 1s/step - loss: 1.6379 - f1_score: 0.4704
concluding trial  1
redoing trial 2. Model was 60.59217071533203MB.


...

,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
0,29,4,0.001683,3.51671,NaN,NaN


trainScore
75/75 [==============================] - 94s 1s/step - loss: 1.0069 - f1_score: 0.9547
devScore
10/10 [==============================] - 12s 1s/step - loss: 1.0666 - f1_score: 0.9272
concluding trial  3
trainScore
75/75 [==============================] - 93s 1s/step - loss: 1.6997 - f1_score: 0.1683
devScore
10/10 [==============================] - 12s 1s/step - loss: 1.6999 - f1_score: 0.1235
concluding trial  4
trainScore
75/75 [==============================] - 95s 1s/step - loss: 1.7588 - f1_score: 0.9976
devScore
10/10 [==============================] - 12s 1s/step - loss: 1.7940 - f1_score: 0.9750
concluding trial  5
trainScore
75/75 [==============================] - 91s 1s/step - loss: 2.5316 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 11s 1s/step - loss: 2.5279 - f1_score: 0.0000e+00
concluding trial  6
trainScore
75/75 [==============================] - 91s 1s/step - loss: 0.5351 - f1_score: 0.9046
devScore
10/10 [======================

,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
5,16,3,0.000182,0.397916,0.997563,0.975000
2,1,8,0.001011,0.014289,0.990558,0.984046
3,25,8,0.001956,3.876973,0.954699,0.927216
7,22,8,0.000129,959.953150,0.904568,0.877204
8,23,7,0.001638,240.640802,0.767368,0.763766
1,22,5,0.003752,551.476787,0.517103,0.470386
4,3,3,0.003486,384.572734,0.168267,0.123498
0,12,9,0.003879,0.039099,0.000000,0.000000
6,14,6,0.008644,0.631308,0.000000,0.000000


Final Model


,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,1,8,0.001011,0.014289,56.34272,0.990558,0.984046,0.98762
